In [156]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
sklearn.set_config(transform_output="pandas")

In [157]:
def split_data(data):
    return (data.drop('lpsa', axis=1), data['lpsa'])

data = pd.read_csv('prostate.data', sep='\t')
data = data.drop(data.columns[[0]], axis=1)
data['Intercept'] = 1

X, Y = split_data(data)
X = X.drop('train', axis=1)
globalmean = X.mean()
globalmean['Intercept'] = 0
globalsigma = X.std()
globalsigma['Intercept'] = 1
globalsigma

for col in data.columns:
    if col == 'lpsa': continue
    if col == 'train': continue
    data[col] = (data[col] - globalmean[col]) / globalsigma[col]

In [158]:
train = data[data['train'] == 'T'].drop('train', axis=1)
test = data[data['train'] == 'F'].drop('train', axis=1)
train.isna().sum()

lcavol       0
lweight      0
age          0
lbph         0
svi          0
lcp          0
gleason      0
pgg45        0
lpsa         0
Intercept    0
dtype: int64

In [159]:
def generate_model(data):
    X, Y = split_data(data)
    # scaler = sklearn.preprocessing.StandardScaler()
    # scaler.fit(X)
    # X = scaler.transform(X)
    # X = (X - globalmean) / globalsigma
    # print(X)
    model = LinearRegression(fit_intercept=False)
    model.fit(X, Y)
    return model
model = generate_model(train)
print(model.coef_)


[ 0.67952814  0.26305307 -0.14146483  0.21014656  0.3052006  -0.28849277
 -0.02130504  0.26695576  2.46493292]


In [160]:
def sigma(data, model):
    X, Y = split_data(data)
    # X = (X - globalmean) / globalsigma
    Yhat = model.predict(X)
    delta = Y - Yhat
    return delta.std(ddof=9)

def MSE(data, model):
    X, Y = split_data(data)
    # X = (X - globalmean) / globalsigma
    Yhat = model.predict(X)
    delta = Y - Yhat
    return delta.std(ddof=0)

model = generate_model(train)
print(f'{sigma(train, model) = }')
print(f'{sigma(test, model) = }')
print(f'{MSE(train, model) = }')
print(f'{MSE(test, model) = }')

X, Y = split_data(train)
Ymean = Y.mean()

X, Y = split_data(test)


print(f'{(Y - Ymean).std(ddof=0) = }')
Ymean = data['lpsa'].mean()
print(f'{(Y - data["lpsa"].mean()).std(ddof=0) = }')

print(len(train))
print(len(test))
print(len(data))

sigma(train, model) = 0.7122860775034966
sigma(test, model) = 0.8613790086829582
MSE(train, model) = 0.662721486039448
MSE(test, model) = 0.7206813842605795
(Y - Ymean).std(ddof=0) = 1.024521002825409
(Y - data["lpsa"].mean()).std(ddof=0) = 1.0245210028254093
67
30
97


In [161]:
X, Y = split_data(train)
# X = (X - globalmean) / globalsigma
Xmatrix = X.values
print(Xmatrix.shape)
XtX = np.matmul(Xmatrix.T, Xmatrix)
print(XtX.shape)
tmp = np.linalg.inv(XtX)
# print(tmp)

v = np.diag(tmp)
v

(67, 9)
(9, 9)


array([0.03160513, 0.0180245 , 0.02024295, 0.02059466, 0.03011133,
       0.04706662, 0.04158214, 0.04651042, 0.01572315])

In [162]:
z = model.coef_ / sigma(train, model) / np.sqrt(v)
print(z)
print(sigma(train, model))
print(np.sqrt(v) * sigma(train, model))

[ 5.36629046  2.75078939 -1.39590898  2.05584563  2.46925518 -1.86691264
 -0.14668121  1.73783972 27.59820312]
0.7122860775034966
[0.12662903 0.09562821 0.10134245 0.10221904 0.12360027 0.15452934
 0.14524723 0.15361357 0.08931498]


In [175]:
params = ['age', 'lcp', 'gleason', 'pgg45']
X, Y = split_data(train)
reducedX = X.drop(params, axis=1)
reducedX

model1 = generate_model(train)
model0 = LinearRegression(fit_intercept=False).fit(reducedX, Y)

RSS0 = ((model0.predict(reducedX) - Y)**2).sum()
RSS1 = ((model1.predict(X) - Y)**2).sum()

print(RSS0, RSS1)

print(len(params))
print(len(train.columns))
F = (RSS0 - RSS1) / len(params) / RSS1 * (len(train) - len(X.columns))
F

32.81499474881555 29.4263844599084
4
10


1.6697548846375199